## 6. Metaclasses and Attributes

### 48 Validate Subclasses with `__init_subclass__`

In [1]:
import logging

In [3]:
from pprint import pprint

In [4]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        global print
        orig_print = print
        print(f'* Running {meta}.__new__ for {name}')
        print('Bases:', bases)
        print = pprint
        print(class_dict)
        print = orig_print
        return type.__new__(meta, name, bases, class_dict)

class MyClass(metaclass=Meta):
    stuff = 123

    def foo(self):
        pass

class MySubclass(MyClass):
    other = 567

    def bar(self):
        pass

* Running <class '__main__.Meta'>.__new__ for MyClass
Bases: ()
{'__module__': '__main__',
 '__qualname__': 'MyClass',
 'foo': <function MyClass.foo at 0x7f3994202040>,
 'stuff': 123}
* Running <class '__main__.Meta'>.__new__ for MySubclass
Bases: (<class '__main__.MyClass'>,)
{'__module__': '__main__',
 '__qualname__': 'MySubclass',
 'bar': <function MySubclass.bar at 0x7f39941fa040>,
 'other': 567}


In [5]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate subclasses of the Polygon class
        if bases:
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

class Polygon(metaclass=ValidatePolygon):
    sides = None  # Must be specified by subclasses

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Triangle(Polygon):
    sides = 3

class Rectangle(Polygon):
    sides = 4

class Nonagon(Polygon):
    sides = 9

assert Triangle.interior_angles() == 180
assert Rectangle.interior_angles() == 360
assert Nonagon.interior_angles() == 1260

In [6]:
try:
    print('Before class')
    
    class Line(Polygon):
        print('Before sides')
        sides = 2
        print('After sides')
    
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-6-d2aa4b235ebc>", line 4, in <module>
    class Line(Polygon):
  File "<ipython-input-5-233d3b29def6>", line 6, in __new__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides


Before class
Before sides
After sides


In [7]:
class BetterPolygon:
    sides = None  # Must be specified by subclasses

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.sides < 3:
            raise ValueError('Polygons need 3+ sides')

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Hexagon(BetterPolygon):
    sides = 6

assert Hexagon.interior_angles() == 720

In [8]:
try:
    print('Before class')
    
    class Point(BetterPolygon):
        sides = 1
    
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-8-784cb391e033>", line 4, in <module>
    class Point(BetterPolygon):
  File "<ipython-input-7-a4dfe6dc9649>", line 7, in __init_subclass__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides


Before class


In [9]:
class ValidateFilled(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate subclasses of the Filled class
        if bases:
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')
        return type.__new__(meta, name, bases, class_dict)

class Filled(metaclass=ValidateFilled):
    color = None  # Must be specified by subclasses

In [10]:
try:
    class RedPentagon(Filled, Polygon):
        color = 'blue'
        sides = 5
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-10-6765422b654a>", line 2, in <module>
    class RedPentagon(Filled, Polygon):
TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases


In [11]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

class Polygon(metaclass=ValidatePolygon):
    is_root = True
    sides = None  # Must be specified by subclasses

class ValidateFilledPolygon(ValidatePolygon):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')
        return super().__new__(meta, name, bases, class_dict)

class FilledPolygon(Polygon, metaclass=ValidateFilledPolygon):
    is_root = True
    color = None  # Must be specified by subclasses

In [12]:
class GreenPentagon(FilledPolygon):
    color = 'green'
    sides = 5

greenie = GreenPentagon()
assert isinstance(greenie, Polygon)

In [13]:
try:
    class OrangePentagon(FilledPolygon):
        color = 'orange'
        sides = 5
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-13-3956f65bed46>", line 2, in <module>
    class OrangePentagon(FilledPolygon):
  File "<ipython-input-11-97393c3d83e4>", line 18, in __new__
    raise ValueError('Fill color must be supported')
ValueError: Fill color must be supported


In [14]:
try:
    class RedLine(FilledPolygon):
        color = 'red'
        sides = 2
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-14-8852d5c29548>", line 2, in <module>
    class RedLine(FilledPolygon):
  File "<ipython-input-11-97393c3d83e4>", line 19, in __new__
    return super().__new__(meta, name, bases, class_dict)
  File "<ipython-input-11-97393c3d83e4>", line 6, in __new__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides


In [15]:
class Filled:
    color = None  # Must be specified by subclasses

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.color not in ('red', 'green', 'blue'):
            raise ValueError('Fills need a valid color')

In [16]:
class RedTriangle(Filled, BetterPolygon):
    color = 'red'
    sides = 3

ruddy = RedTriangle()
assert isinstance(ruddy, Filled)
assert isinstance(ruddy, BetterPolygon)

In [17]:
try:
    print('Before class')
    
    class BlueLine(Filled, BetterPolygon):
        color = 'blue'
        sides = 2
    
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-17-ea9ea21562c6>", line 4, in <module>
    class BlueLine(Filled, BetterPolygon):
  File "<ipython-input-15-1a4287170088>", line 5, in __init_subclass__
    super().__init_subclass__()
  File "<ipython-input-7-a4dfe6dc9649>", line 7, in __init_subclass__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides


Before class


In [18]:
try:
    print('Before class')
    
    class BeigeSquare(Filled, BetterPolygon):
        color = 'beige'
        sides = 4
    
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-18-381b70b27866>", line 4, in <module>
    class BeigeSquare(Filled, BetterPolygon):
  File "<ipython-input-15-1a4287170088>", line 7, in __init_subclass__
    raise ValueError('Fills need a valid color')
ValueError: Fills need a valid color


Before class


In [19]:
class Top:
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Top for {cls}')

class Left(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Left for {cls}')

class Right(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Right for {cls}')

class Bottom(Left, Right):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Bottom for {cls}')

Top for <class '__main__.Left'>
Top for <class '__main__.Right'>
Top for <class '__main__.Bottom'>
Right for <class '__main__.Bottom'>
Left for <class '__main__.Bottom'>


> - 메타클래스의 `__new__` 메서드는 `class` 문의 모든 본문이 처리된 직후에 호출된다.
> - 메타클래스를 사용해 클래스가 정의된 직후이면서 클래스가 생성되기 직전인 시점에 클래스 정의를 변경할 수 있다. 하지만 메타클래스는 원하는 목적을 당성하기에 너무 복잡해지는 경우가 많다.
> - `__init_subclass__`를 사용해 하위 클래스가 정의된 직후, 하위 클래스 타입이 만들어지기 직전에 해당 클래스가 원하는 요건을 잘 갖췄는지 확인하라.
> - `__init_subclass__` 정의 안에서 `super().__init_subclass__()`를 호출해 여러 계층에 걸쳐 클래스를 검증하고 다중 상속을 제대로 처리하도록 하라.